In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
import warnings

# 파일 불러와서 열이름 바꾸기
korean, long_term_frgn, short_term_frgn, resident, card_cnt, card_amt, waste_cnt
(이거 예시는 card_amt임)

In [2]:
df = pd.read_csv("files\\data_preprocessing\\long_term_frgn_preprocessing.csv", encoding='cp949')

# 시계열 피처단위로 변경
df['base_date'] = pd.to_datetime(df['base_date'])

# base_date를 index로 설정
df.index = df['base_date']
df.columns = ['base_date', 'emd_cd', 'long_term_frgn']  
df

,base_date,emd_cd,long_term_frgn
base_date,,,
2018-01-31,2018-01-31,50110250,427986.3363
2018-02-28,2018-02-28,50110250,427240.8119
2018-03-31,2018-03-31,50110250,513638.2631
2018-04-30,2018-04-30,50110250,474601.3510
2018-05-31,2018-05-31,50110250,395262.5483
...,...,...,...
2021-02-28,2021-02-28,50130620,109325.4866
2021-03-31,2021-03-31,50130620,118258.8891
2021-04-30,2021-04-30,50130620,129227.7480


In [3]:
# 필요 없는 열 제거
data = df.drop(['base_date'], 1)    
data

,emd_cd,long_term_frgn
base_date,,
2018-01-31,50110250,427986.3363
2018-02-28,50110250,427240.8119
2018-03-31,50110250,513638.2631
2018-04-30,50110250,474601.3510
2018-05-31,50110250,395262.5483
...,...,...
2021-02-28,50130620,109325.4866
2021-03-31,50130620,118258.8891
2021-04-30,50130620,129227.7480


In [4]:
# 한 행정동의 행만 추출
sub_area = data['emd_cd'] == 50110660

# base_date, card_amt 만 갖는 df 생성
sub_df = data[sub_area]
sub_df = sub_df.drop('emd_cd', 1)
sub_df

,long_term_frgn
base_date,
2018-01-31,538485.6710
2018-02-28,426347.8150
2018-03-31,573962.1490
2018-04-30,511591.9564
2018-05-31,575200.0857
2018-06-30,591935.9388
2018-07-31,584273.7306
2018-08-31,652431.3593
2018-09-30,613235.2351


In [5]:
# series 형으로 변환
sub_df.to_dict('series')

{'long_term_frgn': base_date
 2018-01-31    538485.6710
 2018-02-28    426347.8150
 2018-03-31    573962.1490
 2018-04-30    511591.9564
 2018-05-31    575200.0857
 2018-06-30    591935.9388
 2018-07-31    584273.7306
 2018-08-31    652431.3593
 2018-09-30    613235.2351
 2018-10-31    577318.0170
 2018-11-30    650874.8638
 2018-12-31    582201.0833
 2019-01-31    637037.9877
 2019-02-28    603841.4398
 2019-03-31    791225.6613
 2019-04-30    690914.8586
 2019-05-31    693489.0328
 2019-06-30    770954.6732
 2019-07-31    692131.6583
 2019-08-31    742642.6122
 2019-09-30    694252.2301
 2019-10-31    667143.1521
 2019-11-30    669801.9545
 2019-12-31    688887.6854
 2020-01-31    589050.4792
 2020-02-29    638876.5043
 2020-03-31    568356.2285
 2020-04-30    601752.7211
 2020-05-31    565524.7484
 2020-06-30    573310.5942
 2020-07-31    620232.9132
 2020-08-31    610756.7997
 2020-09-30    534566.6717
 2020-10-31    598866.9918
 2020-11-30    661395.6848
 2020-12-31    643912.7438

# ARIMA

In [6]:
# ARIMA에 적합한 float로 바꾸는 과정
sub_df_float = sub_df[:].astype(np.float)

In [7]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -2.817840
p-value : 0.055770
Critical Values :
	1%: -3.605565
	5%: -2.937069
	10%: -2.606986


In [8]:
# 차분
diff_1 = sub_df_float.diff(periods=1).iloc[1:]

# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
result = adfuller(diff_1)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -11.953006
p-value : 0.000000
Critical Values :
	1%: -3.605565
	5%: -2.937069
	10%: -2.606986


In [9]:
from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df)

 ARIMA(0,0,0)(0,0,0)[1] intercept   : AIC=1058.566, Time=0.01 sec
 ARIMA(0,0,1)(0,0,0)[1] intercept   : AIC=1053.084, Time=0.10 sec
 ARIMA(0,0,2)(0,0,0)[1] intercept   : AIC=1049.669, Time=0.04 sec
 ARIMA(0,0,3)(0,0,0)[1] intercept   : AIC=1050.305, Time=0.06 sec
 ARIMA(0,0,4)(0,0,0)[1] intercept   : AIC=1046.948, Time=0.06 sec
 ARIMA(0,0,5)(0,0,0)[1] intercept   : AIC=1043.758, Time=0.17 sec
 ARIMA(1,0,0)(0,0,0)[1] intercept   : AIC=1046.321, Time=0.03 sec
 ARIMA(1,0,1)(0,0,0)[1] intercept   : AIC=1046.373, Time=0.09 sec
 ARIMA(1,0,2)(0,0,0)[1] intercept   : AIC=1043.425, Time=0.10 sec
 ARIMA(1,0,3)(0,0,0)[1] intercept   : AIC=1042.476, Time=0.15 sec
 ARIMA(1,0,4)(0,0,0)[1] intercept   : AIC=1047.536, Time=0.17 sec
 ARIMA(2,0,0)(0,0,0)[1] intercept   : AIC=1042.983, Time=0.06 sec
 ARIMA(2,0,1)(0,0,0)[1] intercept   : AIC=1044.023, Time=0.10 sec
 ARIMA(2,0,2)(0,0,0)[1] intercept   : AIC=inf, Time=0.22 sec
 ARIMA(2,0,3)(0,0,0)[1] intercept   : AIC=1044.416, Time=0.20 sec
 ARIMA(3,0,0)(0

ARIMA(order=(1, 0, 3), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True)

In [10]:
# ARIMA 모델 
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df.long_term_frgn.values, order=(1,0,3))

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

C:\Users\soeun\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


                              ARMA Model Results                              
Dep. Variable:                      y   No. Observations:                   42
Model:                     ARMA(1, 3)   Log Likelihood                -515.143
Method:                       css-mle   S.D. of innovations          50533.543
Date:                Sat, 11 Sep 2021   AIC                           1042.286
Time:                        04:20:08   BIC                           1052.712
Sample:                             0   HQIC                          1046.107
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        6.17e+05   3.08e+04     20.010      0.000    5.57e+05    6.77e+05
ar.L1.y        0.7928      0.150      5.272      0.000       0.498       1.088
ma.L1.y       -0.5981      0.188     -3.173      0.0

# 예측하기

In [11]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

(array([609296.15722626, 610897.03654854]), array([50533.54337163, 51482.23112249]), array([[510252.23220666, 708340.08224585],
       [509993.71770469, 711800.35539238]]))
